In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/27 22:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [14]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [15]:
df_green.createOrReplaceTempView('green')

In [16]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(*) AS number_records

FROM 
    green
WHERE
    pickup_datetime > '2020-01-01 00:00:00'
GROUP BY 
    zone, hour
""")

In [23]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green/', mode = 'overwrite')

23/02/27 22:40:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [18]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
| 208|2020-01-11 21:00:00| 75.72999999999999|             2|
|  41|2020-01-19 20:00:00|445.98000000000013|            35|
|  75|2020-01-08 17:00:00|1827.6199999999988|           107|
|  74|2020-01-25 16:00:00| 828.4599999999997|            56|
|  74|2020-01-19 20:00:00| 782.8599999999997|            54|
| 255|2020-01-06 21:00:00|            122.49|             8|
|  58|2020-01-30 09:00:00|             18.75|             1|
| 260|2020-01-11 00:00:00|            181.07|            14|
| 264|2020-01-09 06:00:00|              25.8|             1|
| 130|2020-01-29 02:00:00|              68.3|             8|
|  74|2020-01-01 22:00:00|353.15000000000015|            34|
|  26|2020-01-30 08:00:00|142.01999999999998|             4|
| 243|2020-01-20 06:00:00|59.769999999999996|             3|
| 130|2020-01-29 09:00:0

In [19]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [20]:
df_yellow.createOrReplaceTempView('yellow')

In [21]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', pickup_datetime) AS hour, 

    SUM(total_amount) AS amount,
    COUNT(*) AS number_records

FROM 
    yellow
WHERE
    pickup_datetime > '2020-01-01 00:00:00'
GROUP BY 
    zone, hour
""")

In [24]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow/', mode = 'overwrite')

23/02/27 22:40:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/02/27 22:40:45 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [28]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [29]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on = ['hour', 'zone'], how = 'outer')

In [30]:
df_join.show()

+-------------------+----+------------+--------------------+------------------+---------------------+
|               hour|zone|green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+------------------+---------------------+
|2020-01-01 06:00:00|   1|        null|                null|445.28000000000003|                    4|
|2020-01-01 12:00:00|   1|        null|                null|            266.66|                    3|
|2020-01-01 13:00:00|   1|        null|                null|            213.36|                    2|
|2020-01-01 16:00:00|   1|        null|                null|            574.02|                    6|
|2020-01-01 19:00:00|   1|        null|                null|             96.35|                    1|
|2020-01-02 10:00:00|   1|        null|                null|             209.1|                    2|
|2020-01-03 07:00:00|   1|        null|                null|              99.3|   

In [31]:
df_join.write.parquet('data/report/revenue/total')

23/02/27 22:47:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [33]:
df_join = spark.read.parquet('data/report/revenue/total')

In [34]:
df_zones = spark.read.parquet('zones/')

In [37]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.locationid)

In [40]:
df_result.drop('locationid', 'zone').write.parquet('tmp/revenue-zones')

23/02/27 23:29:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
